In [3]:

# 직접 YOLO 클래스를 임포트
from ultralytics import YOLO

# YOLO 모델 초기화
model = YOLO("yolo11x-obb.pt") 
results = model.train(
    data="data.yaml",
    epochs=100,
    batch=16,
    imgsz=256,
    patience=6,
    device="0",
    lr0 = 0.01,
    lrf = 0.1,
    single_cls = True,
    dropout = 0.3   
)



100%|██████████| 113M/113M [00:03<00:00, 34.7MB/s] 


Ultralytics 8.3.5  Python-3.12.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
engine\trainer: task=obb, mode=train, model=yolo11x-obb.pt, data=data.yaml, epochs=100, time=None, patience=6, batch=16, imgsz=256, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.3, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tru

train: Scanning E:\Workspace\SatellieteIMGDetection\datasets\labels\train.cache... 357 images, 0 backgrounds, 0 corrupt: 100%|██████████| 357/357 [00:00<?, ?it/s]
val: Scanning E:\Workspace\SatellieteIMGDetection\datasets\labels\val.cache... 37 images, 0 backgrounds, 0 corrupt: 100%|██████████| 37/37 [00:00<?, ?it/s]


Plotting labels to runs\obb\train10\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 173 weight(decay=0.0), 183 weight(decay=0.0005), 182 bias(decay=0.0)
Image sizes 256 train, 256 val
Using 8 dataloader workers
Logging results to runs\obb\train10
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      3.99G      2.379      2.691      1.358        217        256: 100%|██████████| 23/23 [00:04<00:00,  4.67it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  7.54it/s]

...

      9/100      4.26G      2.169      1.352      1.231        754        256: 100%|██████████| 23/23 [00:03<00:00,  7.19it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  9.67it/s]

                   all         37       1653      0.335      0.217      0.152     0.0522
EarlyStopping: Training stopped early as no improvement observed in last 6 epochs. Best results observed at epoch 3, best model saved as best.pt.
To update EarlyStopping(patience=6) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



9 epochs completed in 0.014 hours.
Optimizer stripped from runs\obb\train10\weights\last.pt, 118.2MB
Optimizer stripped from runs\obb\train10\weights\best.pt, 118.2MB

Validating runs\obb\train10\weights\best.pt...
Ultralytics 8.3.5  Python-3.12.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
YOLO11x-obb summary (fused): 483 layers, 58,736,758 parameters, 0 gradients, 202.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.49it/s]


                   all         37       1653      0.379      0.226      0.188     0.0565
Speed: 0.1ms preprocess, 5.7ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs\obb\train10


In [32]:
# YOLO 모델의 Detect 모듈을 제외한 백본만 추출
backbone = nn.Sequential(*list(yolo.model.model.children())[:-1])
backbone

Sequential(
  (0): Conv(
    (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (1): Conv(
    (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (2): C3k2(
    (cv1): Conv(
      (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (cv2): Conv(
      (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (m): ModuleList(
      (0): C3k(
        (cv1): Conv(
      

In [34]:
head = nn.Sequential(list(yolo.model.model.children())[-1])
head

Sequential(
  (0): Detect(
    (cv2): ModuleList(
      (0): Sequential(
        (0): Conv(
          (conv): Conv2d(256, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      )
      (1-2): 2 x Sequential(
        (0): Conv(
          (conv): Conv2d(512, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(64, 64,

In [1]:
import torch
import torch.nn as nn
from ultralytics import YOLO

# YOLO 모델 불러오기 (YOLOv5 예시)
class YOLOv5WithCustomHead(nn.Module):
    def __init__(self, yolo):
        super(YOLOv5WithCustomHead, self).__init__()
        self.yolo = yolo.model  # YOLO 모델의 모든 레이어
        
        # YOLO의 중간층에서 사용할 레이어 인덱스를 확인해야 함
        self.custom_head = nn.Sequential(
            nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),  # Global Average Pooling
            nn.Flatten(),
            nn.Linear(128, 10)  # 예시로 10개 클래스를 분류하는 헤드
        )

    def forward(self, x):
        outputs = []  # 여러 텐서를 저장할 리스트
        
        # 중간 피처 맵을 추출하는 부분
        for i, layer in enumerate(self.yolo.model):
            x = layer(x)
            if isinstance(x, list):  # x가 리스트인 경우
                x = torch.cat(x, dim=1)  # 리스트의 텐서를 결합
            if i == 9:  # 예시로 9번째 레이어 출력 사용
                feature_map = x  # 중간 피처 맵 추출
                outputs.append(feature_map)  # 필요한 피처 맵 저장

        # 기존 YOLO 아웃풋 (객체 탐지)
        yolo_output = x

        # 새로운 헤드에서 중간 피처 맵 사용
        custom_output = self.custom_head(feature_map)
        
        return yolo_output, custom_output

# 기존 YOLOv5 모델 불러오기
yolo = YOLO('yolo11m.pt')

# 커스텀 모델 생성 (YOLOv5에 새로운 헤드 추가)
model_with_custom_head = YOLOv5WithCustomHead(yolo)

# 예시 입력 데이터
input_data = torch.randn(1, 3, 640, 640)

# 모델 실행
yolo_output, custom_output = model_with_custom_head(input_data)

# 결과 출력
print("YOLO Output Shape:", yolo_output.shape)
print("Custom Head Output Shape:", custom_output.shape)



TypeError: cat() received an invalid combination of arguments - got (Tensor, int), but expected one of:
 * (tuple of Tensors tensors, int dim, *, Tensor out)
 * (tuple of Tensors tensors, name dim, *, Tensor out)


In [9]:
yolo.model

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
   